# streaming

> SSE streaming utilities for creating and managing Server-Sent Events streams

In [ ]:
#| default_exp core.streaming

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import asyncio
import json
from typing import Optional, Dict, Any, AsyncIterator, Union, List, Callable
from datetime import datetime
from dataclasses import dataclass

from fasthtml.common import sse_message, FT

# Import broadcast components
from cjm_fasthtml_sse.core.broadcast import (
    SSEBroadcastManager, 
    BroadcastMessage,
    SSEConnection
)

## SSE Message Formatting

In [ ]:
#| export
def format_sse_message(
    data: Union[str, Dict[str, Any], Any],
    event: Optional[str] = 'message',  # Optional event type
    id: Optional[str] = None,  # Optional event ID
    retry: Optional[int] = None  # Optional retry interval in milliseconds
) -> str:  # Formatted SSE message string
    "Format data as a Server-Sent Event message."

    if isinstance(data, FT):
        return sse_message(data, event=event)
    
    lines = []
    
    if event:
        lines.append(f"event: {event}")
    
    if id:
        lines.append(f"id: {id}")
    
    if retry is not None:
        lines.append(f"retry: {retry}")
    
    # Convert data to string if necessary
    if isinstance(data, str):
        data_str = data
    else:
        try:
            data_str = json.dumps(data)
        except (TypeError, ValueError):
            data_str = str(data)
    
    # Handle multi-line data
    for line in data_str.split('\n'):
        lines.append(f"data: {line}")
    
    # SSE messages end with double newline
    return '\n'.join(lines) + '\n\n'

In [ ]:
#| export
def format_sse_comment(
    comment: str  # Comment text
) -> str:  # Formatted SSE comment
    "Format a comment for SSE (typically used for heartbeats)."

    if isinstance(comment, FT):
        return sse_message(comment)
    
    return f": {comment}\n\n"

## SSE Stream Generator

In [ ]:
#| export
@dataclass
class SSEStreamConfig:
    """
    Configuration for SSE streams.
    """
    heartbeat_interval: Optional[float] = 30.0 # Seconds between heartbeat messages (None to disable)
    reconnect_time: Optional[int] = 3000 # Suggested reconnect time for clients in milliseconds
    timeout: Optional[float] = None # Maximum time to wait for messages before closing stream
    initial_event: Optional[Dict[str, Any]] = None # Event to send when stream starts
    closing_event: Optional[Dict[str, Any]] = None # Event to send when stream closes
    debug: bool = False # Enable debug logging

In [ ]:
#| export
async def sse_broadcast_stream(
    manager: SSEBroadcastManager,  # The SSEBroadcastManager to connect to
    client_metadata: Optional[Dict[str, Any]] = None,
    config: Optional[SSEStreamConfig] = None,  # Optional stream configuration
    message_filter: Optional[Callable[[BroadcastMessage], bool]] = None
) -> AsyncIterator[str]:  # Async iterator yielding formatted SSE messages
    """Create an SSE stream that receives messages from a broadcast manager. This is the main function for creating SSE endpoints that receive broadcast messages and format them for SSE delivery."""
    config = config or SSEStreamConfig()
    
    def log(
        msg: str  # Message to log if debug is enabled
    ):
        """Internal debug logging function"""
        if config.debug:
            print(f"[SSE Stream] {msg}")
    
    # Use connection context manager
    async with SSEConnection(manager, metadata=client_metadata) as queue:
        log(f"Client connected (active: {manager.get_connection_count()})")
        
        try:
            # Send initial event if configured
            if config.initial_event:
                yield format_sse_message(
                    config.initial_event,
                    event="connect"
                )
            
            # Send suggested reconnect time
            if config.reconnect_time:
                yield format_sse_message(
                    {"status": "connected"},
                    retry=config.reconnect_time
                )
            
            # Main message loop
            while True:
                try:
                    # Wait for message with heartbeat timeout
                    timeout = config.heartbeat_interval or config.timeout
                    
                    message = await asyncio.wait_for(
                        queue.get(),
                        timeout=timeout
                    )
                    
                    # Apply filter if provided
                    if message_filter and not message_filter(message):
                        continue
                    
                    # Format and yield the message
                    yield format_sse_message(
                        message.to_dict(),
                        event=message.type
                    )
                    
                except asyncio.TimeoutError:
                    if config.heartbeat_interval:
                        # Send heartbeat
                        yield format_sse_comment(
                            f"heartbeat {datetime.now().isoformat()}"
                        )
                    elif config.timeout:
                        # Timeout reached, close stream
                        log("Stream timeout reached")
                        break
                        
        except asyncio.CancelledError:
            log("Stream cancelled")
            raise
        except Exception as e:
            log(f"Stream error: {e}")
            raise
        finally:
            # Send closing event if configured
            if config.closing_event:
                yield format_sse_message(
                    config.closing_event,
                    event="close"
                )
            log(f"Client disconnected (active: {manager.get_connection_count() - 1})")

## Direct SSE Stream Utilities

In [ ]:
#| export
async def sse_generator(
    data_source: Union[AsyncIterator, Callable],
    interval: float = 1.0,  # Seconds between data checks (if using callable)
    event_type: Optional[str] = None,  # Optional event type for all messages
    heartbeat: Optional[float] = 30.0  # Heartbeat interval in seconds (None to disable)
) -> AsyncIterator[str]:  # Async iterator yielding formatted SSE messages
    """Create an SSE stream from a data source. This is useful for creating SSE streams from custom data sources without using the broadcast manager."""
    last_heartbeat = datetime.now().timestamp()
    
    try:
        # Handle async iterator
        if hasattr(data_source, '__aiter__'):
            async for data in data_source:
                yield format_sse_message(data, event=event_type)
                
                # Check for heartbeat
                if heartbeat:
                    now = datetime.now().timestamp()
                    if now - last_heartbeat > heartbeat:
                        yield format_sse_comment(f"heartbeat {datetime.now().isoformat()}")
                        last_heartbeat = now
        
        # Handle callable
        elif callable(data_source):
            while True:
                # Get data from callable
                if asyncio.iscoroutinefunction(data_source):
                    data = await data_source()
                else:
                    data = data_source()
                
                # Send data if available
                if data is not None:
                    yield format_sse_message(data, event=event_type)
                
                # Wait for next interval
                await asyncio.sleep(interval)
                
                # Check for heartbeat
                if heartbeat:
                    now = datetime.now().timestamp()
                    if now - last_heartbeat > heartbeat:
                        yield format_sse_comment(f"heartbeat {datetime.now().isoformat()}")
                        last_heartbeat = now
        else:
            raise TypeError("data_source must be an async iterator or callable")
            
    except asyncio.CancelledError:
        # Send close event on cancellation
        yield format_sse_message({"status": "closed"}, event="close")
        raise

## OOB (Out-of-Band) Message Utilities

In [ ]:
#| export
@dataclass
class OOBUpdate:
    """
    Represents an out-of-band update for HTMX.
    """
    target_id: str # The ID of the element to update
    content: str # The HTML content to swap in
    swap_type: str = "innerHTML" # The type of swap (innerHTML, outerHTML, etc.)
    
    def to_html(
        self
    ) -> str:  # HTML string with hx-swap-oob attribute for HTMX
        """Convert to HTML with hx-swap-oob attribute"""
        # For HTMX OOB, we need to wrap content with the target element
        if self.swap_type == "outerHTML":
            # Content should already include the element with ID
            return self.content
        else:
            # Wrap content in a div with OOB swap directive
            return f'<div id="{self.target_id}" hx-swap-oob="{self.swap_type}">{self.content}</div>'

In [ ]:
#| export
def format_oob_updates(
    updates: List[OOBUpdate]  # List of OOB updates to format
) -> str:  # Combined HTML string with all OOB updates
    "Format multiple OOB updates into a single HTML string."
    if not updates:
        return ""
    
    html_parts = [update.to_html() for update in updates]
    return ''.join(html_parts)

In [ ]:
#| export
def create_oob_message(
    updates: List[OOBUpdate],  # List of OOB updates
    event: str = "message"  # SSE event type
) -> str:  # Formatted SSE message with OOB HTML
    "Create an SSE message containing OOB updates for HTMX."
    html = format_oob_updates(updates)
    return format_sse_message(html, event=event)

## Example Usage

In [ ]:
# Example: Basic SSE message formatting
print("Basic message:")
print(format_sse_message({"hello": "world"}))

print("Message with event type:")
print(format_sse_message(
    {"count": 42},
    event="update",
    id="msg-1"
))

print("Heartbeat comment:")
print(format_sse_comment("ping"))

Basic message:
event: message
data: {"hello": "world"}


Message with event type:
event: update
id: msg-1
data: {"count": 42}


Heartbeat comment:
: ping




In [ ]:
# Example: Using broadcast stream
async def example_broadcast_stream():
    manager = SSEBroadcastManager(debug=True)
    
    # Simulate sending some messages in background
    async def send_messages():
        await asyncio.sleep(0.1)
        await manager.broadcast("notification", {"msg": "Hello"})
        await asyncio.sleep(0.1)
        await manager.broadcast("update", {"value": 123})
    
    # Start message sender
    asyncio.create_task(send_messages())
    
    # Create stream and collect messages
    messages = []
    config = SSEStreamConfig(
        heartbeat_interval=None,  # Disable heartbeat for test
        timeout=0.5,  # Short timeout for test
        initial_event={"status": "ready"},
        debug=True
    )
    
    async for msg in sse_broadcast_stream(manager, config=config):
        messages.append(msg)
        if len(messages) >= 3:  # Initial + 2 broadcasts
            break
    
    print(f"Received {len(messages)} messages")
    for i, msg in enumerate(messages[:3]):
        print(f"\nMessage {i+1}:")
        print(msg[:100] + "..." if len(msg) > 100 else msg)

await example_broadcast_stream()

[SSEBroadcastManager] Connection registered. Total connections: 1
[SSE Stream] Client connected (active: 1)
[SSEBroadcastManager] Broadcast to 1/1 clients
Received 3 messages

Message 1:
event: connect
data: {"status": "ready"}



Message 2:
event: message
retry: 3000
data: {"status": "connected"}



Message 3:
event: notification
data: {"type": "notification", "data": {"msg": "Hello"}, "timestamp": "2025-08-2...
[SSE Stream] Client disconnected (active: 0)
[SSEBroadcastManager] Connection unregistered. Total connections: 0
[SSEBroadcastManager] No active connections to broadcast to


In [ ]:
# Example: Direct SSE generator from data source
async def example_direct_generator():
    # Create a simple data source
    counter = 0
    def get_data():
        nonlocal counter
        counter += 1
        return {"count": counter} if counter <= 3 else None
    
    # Generate SSE stream
    messages = []
    async for msg in sse_generator(
        get_data,
        interval=0.1,
        event_type="counter",
        heartbeat=None
    ):
        messages.append(msg)
        if len(messages) >= 3:
            break
    
    print("Generated messages:")
    for msg in messages:
        print(msg)

await example_direct_generator()

Generated messages:
event: counter
data: {"count": 1}


event: counter
data: {"count": 2}


event: counter
data: {"count": 3}




In [ ]:
# Example: OOB updates
updates = [
    OOBUpdate("status", "<span class='badge'>Active</span>"),
    OOBUpdate("counter", "<strong>42</strong>"),
    OOBUpdate("progress", "<div class='progress' value='75'></div>", "outerHTML")
]

print("OOB Updates HTML:")
print(format_oob_updates(updates))

print("\nOOB SSE Message:")
print(create_oob_message(updates))

OOB Updates HTML:
<div id="status" hx-swap-oob="innerHTML"><span class='badge'>Active</span></div><div id="counter" hx-swap-oob="innerHTML"><strong>42</strong></div><div class='progress' value='75'></div>

OOB SSE Message:
event: message
data: <div id="status" hx-swap-oob="innerHTML"><span class='badge'>Active</span></div><div id="counter" hx-swap-oob="innerHTML"><strong>42</strong></div><div class='progress' value='75'></div>




## Testing

In [ ]:
# Test: SSE message formatting
def test_sse_formatting():
    # Test basic message
    msg = format_sse_message("hello")
    assert "data: hello" in msg
    assert msg.endswith("\n\n")
    
    # Test with event
    msg = format_sse_message({"test": 1}, event="update")
    assert "event: update" in msg
    assert 'data: {"test": 1}' in msg
    
    # Test comment
    comment = format_sse_comment("heartbeat")
    assert comment == ": heartbeat\n\n"
    
    print("✓ SSE formatting tests passed")

test_sse_formatting()

✓ SSE formatting tests passed


In [ ]:
# Test: OOB updates
def test_oob_updates():
    update = OOBUpdate("test-id", "<p>Content</p>")
    html = update.to_html()
    assert 'id="test-id"' in html
    assert 'hx-swap-oob="innerHTML"' in html
    assert "<p>Content</p>" in html
    
    # Test multiple updates
    updates = [
        OOBUpdate("id1", "content1"),
        OOBUpdate("id2", "content2")
    ]
    combined = format_oob_updates(updates)
    assert 'id="id1"' in combined
    assert 'id="id2"' in combined
    
    print("✓ OOB update tests passed")

test_oob_updates()

✓ OOB update tests passed


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()